# Balanceo de clases

## Introducción. Preparación de librerias

Usaremos la libreria de textattack para construir nuestro nuevo dataset que tenga las dos clases balanceadas

In [ ]:
!pip install textattack[tensorflow]

Vamos a usar herramientas preentrenadas para realizar las transformaciones en nuestro conjunto de datos

Todas las disponibles están disponibles en el siguiente enlace:

https://textattack.readthedocs.io/en/latest/3recipes/augmenter_recipes.html

Resumen:
Esta librería incorpora diferentes soluciones para el data augmentation. Las que incluye son:

1) WordNetAugmenter: replacing words with synonyms

2) EmbeddingAugmenter: replacing words with neighbors in the counter-fitted embedding space

3) CharSwapAugmenter: It augments text by substituting, deleting, inserting, and swapping adjacent characters

4) EasyDataAugmenter: augments text with a combination of word insertions, substitutions, and deletions.

5) CheckListAugmenter: It augments text by contraction/extension and by substituting names, locations and numbers.

6) CLAREAugmenter: It augments text by replacing, inserting, and merging with a pre-trained masked language model.

Nosotros vamos a usar por ejemplo EmbeddingAugmenter, un ejemplo sencillo:

In [ ]:
# Import, debe cargar más datos
from textattack.augmentation import EmbeddingAugmenter

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
from textattack.augmentation import EmbeddingAugmenter
augmenter = EmbeddingAugmenter()
s = 'What I cannot create, I do not understand.'
augmenter.augment(s)

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:36<00:00, 13.1MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp8mrovtk0.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


['What I notable create, I do not understand.']

## Carga de datos

Haremos un primer ejemplo con los datos de nuestro problema para comprobar que nos sirve



In [ ]:
# Imports
import pandas as pd
import numpy as np

In [ ]:
# Load
df = pd.read_csv("/content/sexism_data.csv")

df = df[['text', 'sexist']]
df['sexist'].value_counts()

False    11822
True      1809
Name: sexist, dtype: int64

In [ ]:
# Imports de preprocesamiento de textos
import re
import string

In [ ]:
# Auxiliar function for remove all emoji characters
# https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def removeEmoji(text):
    emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002500-\U00002BEF"  # chinese char
      u"\U00002702-\U000027B0"
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      u"\U0001f926-\U0001f937"
      u"\U00010000-\U0010ffff"
      u"\u2640-\u2642"
      u"\u2600-\u2B55"
      u"\u200d"
      u"\u23cf"
      u"\u23e9"
      u"\u231a"
      u"\ufe0f"  # dingbats
      u"\u3030"
      "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)

# Preprocessor function
def preprocessor2(title_text):

    # Primero, segundo y cuarto, eliminiación de enlaces, menciones y hashtags - Incluimos eliminar espacios y caracteres raros
    title_text = title_text.apply(lambda x: re.sub(r'((https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b)|(MENTION[0-9]*)|(#[a-zA-Z0-9]*)', ' ', x, flags=re.MULTILINE))
    title_text = title_text.str.replace('\n',' ')
    title_text = title_text.str.replace('\t',' ')

    # Quitamos simbolos de puntuacion
    title_text = title_text.str.translate(str.maketrans('', '', string.punctuation))

    # Eliminamos los restos que no hayan sido eliminados y algunas expresiones que no aportan informacion
    title_text = title_text.apply(lambda x: re.sub(r'([^0-9a-zA-Z:,\s]+)|(rt)|(lol)|(lmao)|(lmfao)', '', x, flags=re.MULTILINE))

    # Despues de eliminar los simbolos de puntuacion, eliminamos los números
    title_text = title_text.apply(lambda x: re.sub(r'([0-9]+)', ' ', x, flags=re.MULTILINE))
    # Tercero, eliminiación de emojis
    title_text = title_text.apply(lambda x: removeEmoji(x))
    # Quitamos palabras sueltas con letras repetidas
    title_text = title_text.apply(lambda x: re.sub(r'(.)\1{3,}', ' ', x, flags=re.MULTILINE))

    # Ponemos todo en minusculas y aplicamos trim
    title_text = title_text.str.lower()
    title_text = title_text.str.strip()
    # Quitamos espacios sobrantes
    title_text = title_text.apply(lambda x: ' '.join([y for y in x.split(' ') if y != '']))

    return title_text

In [ ]:
# Aplicamos a nuestro dataset. Eliminamos twits vacios y que no tengan mucho significado
min_lenght = 10

# Eliminacion de palabras sueltas
df['text'] = preprocessor2(df['text'])
df = df[df['text'].str.len()>min_lenght]

In [ ]:
# Datos tras el filtrado
df['sexist'].value_counts()

False    11218
True      1807
Name: sexist, dtype: int64

Para este ejemplo vamos a usar WordNetAugmenter, el cual nos parece más interesante para este ejemplo

In [ ]:
from textattack.augmentation import WordNetAugmenter
augmenter = WordNetAugmenter(pct_words_to_swap=0.4, transformations_per_example=3)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
s = df['text'][0]
saug = augmenter.augment(s)

In [ ]:
s, saug

('i didnt even know random was an option',
 ['i didnt still live random was an choice',
  'i didnt tied bed random was an alternative',
  'i didnt tied screw random was an choice'])

Podemos comprobar como cambia la frase un poco de forma que podamos aumentar de forma considerable los ejemplos de la clase minoritaría para realizar un entrenamiento más equilibrado


## Aplicación a nuestro dataset

Ampliaremos nuestro dataset conforme nos haga falta, habiendo estudiado el dataset podemos comprobar como nos hará falta como 6 veces más de ejemplos en nuestra clase minoritaria para conseguir los resultados que queremos

Usaremos de referencia que encontramos en el siguiente GitHub:

https://github.com/kothiyayogesh/medium-article-code/tree/master/How%20I%20dealt%20with%20Imbalanced%20text%20dataset


In [ ]:
# Imports
import math
import tqdm
import operator

Usaremos la versión del código expuesto para aplicar el aumento de datos

In [ ]:
# Controlamos la variable que queremos balancear
sexist_count = df.sexist.value_counts().to_dict()
# Guardamos el maximo de las clases, en este caso la falsa
max_sexist_count = max(sexist_count.items(), key=operator.itemgetter(1))[1]

sexist_count, max_sexist_count

({False: 11220, True: 1807}, 11220)

In [ ]:
newdf1 = pd.DataFrame()
# Aplicamos para cada clase, en nuestro caso solo usaremos la clase positiva
for sexist, count in sexist_count.items() :
  
    count_diff = max_sexist_count - count    ## Diferencia entre las clases
    multiplication_count = math.ceil((count_diff)/count)  ## Calculamos con una multiplicacion el número de veces que se va a aplicar data augmentation a cada frase

    # Si el indice de multiplicación es mayor que 0, aplicamos la función de aumento de datos
    if (multiplication_count > 0) :
      # Creación de nuestro data augmenter
      augmenter = WordNetAugmenter(pct_words_to_swap=0.4, transformations_per_example=multiplication_count+ int(multiplication_count/2)) # creamos un 50% mas de ejemplos y luego los elegimos aleatoriamente

      # Variables de control
      old_text_df = pd.DataFrame()
      new_text_df = pd.DataFrame()  

      # Recorremos todos los textos que tenemos disponibles
      for text in tqdm.tqdm(df[df["sexist"] == sexist]["text"]) :
        ## Asignamos el ejemplo base al nuevo dataframe
        dummy1 = pd.DataFrame([text], columns=['text'])
        dummy1["sexist"] = sexist
        old_text_df = old_text_df.append(dummy1)

        ## Creamos con data augmnentation las nuevas frases
        new_messages = augmenter.augment(text)
        dummy2 = pd.DataFrame(new_messages, columns=['text'])
        # Le ponemos la misma etiqueta
        dummy2["sexist"] = sexist
        new_text_df = new_text_df.append(dummy2)

      ## Select random data points from augmented data
      new_text_df=new_text_df.take(np.random.permutation(len(new_text_df))[:count_diff])
      
      ## Merge existing and augmented data points
      newdf1 = newdf1.append([old_text_df,new_text_df])

    # Si la clase no es minoritaria no hace nada
    else :
        newdf1 = newdf1.append(df[df["sexist"] == sexist])


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
100%|██████████| 1807/1807 [24:47<00:00,  1.21it/s]


In [ ]:
df.shape

(13023, 2)

In [ ]:
newdf1['sexist'].value_counts()

True     11220
False    11216
Name: sexist, dtype: int64

In [ ]:
newdf1.to_csv('/content/sexism_data_balanced.csv', index=False)